In [1]:
# ============================================================
# TensorFlow-only implementation
# GPR (TF Probability) + PINN-style Neural Network
# Two cases:
#   (A) Without Langmuir_ADS
#   (B) With Langmuir_ADS
# Target: EXP_ADS (mL/g)
# ============================================================

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

tfd = tfp.distributions
tfk = tf.keras

# ------------------------------------------------------------
# Data
# ------------------------------------------------------------
dataset=pd.read_csv("dataset2.csv")
df = dataset.copy()

y = df["EXP_ADS (mL/g)"].values.reshape(-1, 1)

X_no_lang = df[["Pressure (kPa)"]].values
X_with_lang = df[["Pressure (kPa)", "Langmuir_ADS (mL/g)"]].values

Xn_tr, Xn_te, y_tr, y_te = train_test_split(
    X_no_lang, y, test_size=0.25, random_state=42
)

Xl_tr, Xl_te, _, _ = train_test_split(
    X_with_lang, y, test_size=0.25, random_state=42
)

# Scaling
sc_Xn = StandardScaler()
sc_Xl = StandardScaler()
sc_y = StandardScaler()

Xn_tr = sc_Xn.fit_transform(Xn_tr)
Xn_te = sc_Xn.transform(Xn_te)

Xl_tr = sc_Xl.fit_transform(Xl_tr)
Xl_te = sc_Xl.transform(Xl_te)

y_tr_s = sc_y.fit_transform(y_tr)
y_te_s = sc_y.transform(y_te)

# ------------------------------------------------------------
# Evaluation helper
# ------------------------------------------------------------
def evaluate(y_true, y_pred, name):
    return {
        "Model": name,
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAE": mean_absolute_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred)
    }

results = []

# ============================================================
# 1. Gaussian Process Regression (TensorFlow Probability)
# ============================================================

def train_gpr(Xtr, ytr, Xte):
    kernel = tfp.math.psd_kernels.ExponentiatedQuadratic()
    gpr = tfd.GaussianProcessRegressionModel(
        kernel=kernel,
        index_points=Xte,
        observation_index_points=Xtr,
        observations=ytr[:, 0],
        observation_noise_variance=1e-4
    )
    return gpr.mean().numpy().reshape(-1, 1)

# --- GPR without Langmuir ---
y_pred = train_gpr(Xn_tr, y_tr_s, Xn_te)
y_pred = sc_y.inverse_transform(y_pred)

results.append(
    evaluate(y_te, y_pred, "GPR (no Langmuir)")
)

# --- GPR with Langmuir ---
y_pred = train_gpr(Xl_tr, y_tr_s, Xl_te)
y_pred = sc_y.inverse_transform(y_pred)

results.append(
    evaluate(y_te, y_pred, "GPR (with Langmuir)")
)

# ============================================================
# 2. PINN-style Neural Network (TensorFlow)
# Physics constraint: dV/dP >= 0 (monotonic adsorption)
# ============================================================

class PINN(tfk.Model):
    def __init__(self, in_dim):
        super().__init__()
        self.net = tfk.Sequential([
            tfk.layers.Dense(32, activation="tanh"),
            tfk.layers.Dense(32, activation="tanh"),
            tfk.layers.Dense(1)
        ])

    def call(self, x):
        return self.net(x)

def train_pinn(Xtr, ytr, Xte, epochs=3000, lambda_phys=0.1):
    model = PINN(Xtr.shape[1])
    opt = tfk.optimizers.Adam(1e-3)
    mse = tfk.losses.MeanSquaredError()

    Xtr_tf = tf.convert_to_tensor(Xtr, dtype=tf.float32)
    ytr_tf = tf.convert_to_tensor(ytr, dtype=tf.float32)

    for _ in range(epochs):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(Xtr_tf)
            y_pred = model(Xtr_tf)
            loss_data = mse(ytr_tf, y_pred)

            grads = tape.gradient(y_pred, Xtr_tf)[:, 0]
            loss_phys = tf.reduce_mean(tf.nn.relu(-grads))

            loss = loss_data + lambda_phys * loss_phys

        opt.apply_gradients(
            zip(tape.gradient(loss, model.trainable_variables),
                model.trainable_variables)
        )

    y_out = model(tf.convert_to_tensor(Xte, dtype=tf.float32)).numpy()
    return y_out

# --- PINN without Langmuir ---
y_pred = train_pinn(Xn_tr, y_tr_s, Xn_te)
y_pred = sc_y.inverse_transform(y_pred)

results.append(
    evaluate(y_te, y_pred, "PINN (no Langmuir)")
)

# --- PINN with Langmuir ---
y_pred = train_pinn(Xl_tr, y_tr_s, Xl_te)
y_pred = sc_y.inverse_transform(y_pred)

results.append(
    evaluate(y_te, y_pred, "PINN (with Langmuir)")
)

# ------------------------------------------------------------
# Results
# ------------------------------------------------------------
results_df = pd.DataFrame(results)
results_df


,Model,RMSE,MAE,R2
0,GPR (no Langmuir),3.705763,2.817680,0.418241
1,GPR (with Langmuir),1.364906,1.041090,0.921079
2,PINN (no Langmuir),3.744236,2.858372,0.406099
3,PINN (with Langmuir),1.391752,1.154046,0.917944
